In [3]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import seaborn as sns

In [4]:
ourschema="""
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates" (
    "id" INTEGER PRIMARY KEY  NOT NULL ,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR,
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
"""

In [5]:
from sqlite3 import dbapi2 as sq3
import os
PATHSTART="."
def get_db(dbfile):
    sqlite_db = sq3.connect(os.path.join(PATHSTART,dbfile))
    return sqlite_db


In [6]:
def init_db(dbfile,schema):
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    db.commit()
    return db

In [7]:
dfcand=pd.read_csv("./candidates.txt",sep='|')
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


In [8]:
dfcwci=pd.read_csv("./contributors_with_candidate_id.txt",sep="|")
dfcwci.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16


In [16]:
dfcwci.columns
dfcwci=dfcwci.drop("id",axis=1)

In [17]:
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16


In [18]:
db=init_db("cancont.db",ourschema)

Add Pandas Dataframe to Database

In [28]:
dfcand.to_sql("candidates",db,if_exists="append",index=False)

In [29]:
dfcwci.to_sql("contributors",db,if_exists="append",index=False)

In [17]:
dfcwci.shape

(175, 11)

In [18]:
sel="select * from candidates;"
c=db.cursor().execute(sel)

In [19]:
c.fetchall()

[(16, u'Mike', u'Huckabee', None, u'R'),
 (20, u'Barack', u'Obama', None, u'D'),
 (22, u'Rudolph', u'Giuliani', None, u'R'),
 (24, u'Mike', u'Gravel', None, u'D'),
 (26, u'John', u'Edwards', None, u'D'),
 (29, u'Bill', u'Richardson', None, u'D'),
 (30, u'Duncan', u'Hunter', None, u'R'),
 (31, u'Dennis', u'Kucinich', None, u'D'),
 (32, u'Ron', u'Paul', None, u'R'),
 (33, u'Joseph', u'Biden', None, u'D'),
 (34, u'Hillary', u'Clinton', u'R.', u'D'),
 (35, u'Mitt', u'Romney', None, u'R'),
 (36, u'Samuel', u'Brownback', None, u'R'),
 (37, u'John', u'McCain', None, u'R'),
 (38, u'Tom', u'Tancredo', None, u'R'),
 (39, u'Christopher', u'Dodd', u'J.', u'D'),
 (41, u'Fred', u'Thompson', u'D.', u'R')]

In [19]:
rem="Delete from candidates;"
c=db.cursor().execute(rem)
db.commit()

In [20]:
c.fetchall()

[]

In [21]:
ins="insert into candidates (id,first_name,last_name,middle_name, party) values (?,?,?,?,?)"
with open("candidates.txt") as fd:
    slines = [l.strip().split("|") for l in fd.readlines()]
    for line in slines[1:]:
        theid, first_name, last_name, middle_name, party = line
        print theid, first_name, last_name, middle_name, party
        valstoinsert = (int(theid),first_name,last_name,middle_name,party)
        print ins, valstoinsert
        db.cursor().execute(ins,valstoinsert)

33 Joseph Biden  D
insert into candidates (id,first_name,last_name,middle_name, party) values (?,?,?,?,?) (33, 'Joseph', 'Biden', '', 'D')
36 Samuel Brownback  R
insert into candidates (id,first_name,last_name,middle_name, party) values (?,?,?,?,?) (36, 'Samuel', 'Brownback', '', 'R')
34 Hillary Clinton R. D
insert into candidates (id,first_name,last_name,middle_name, party) values (?,?,?,?,?) (34, 'Hillary', 'Clinton', 'R.', 'D')
39 Christopher Dodd J. D
insert into candidates (id,first_name,last_name,middle_name, party) values (?,?,?,?,?) (39, 'Christopher', 'Dodd', 'J.', 'D')
26 John Edwards  D
insert into candidates (id,first_name,last_name,middle_name, party) values (?,?,?,?,?) (26, 'John', 'Edwards', '', 'D')
22 Rudolph Giuliani  R
insert into candidates (id,first_name,last_name,middle_name, party) values (?,?,?,?,?) (22, 'Rudolph', 'Giuliani', '', 'R')
24 Mike Gravel  D
insert into candidates (id,first_name,last_name,middle_name, party) values (?,?,?,?,?) (24, 'Mike', 'Gravel', 

In [22]:
def make_query(sel):
    c = db.cursor().execute(sel)
    return c.fetchall()

In [24]:
make_query("select * from candidates;")

[(16, u'Mike', u'Huckabee', u'', u'R'),
 (20, u'Barack', u'Obama', u'', u'D'),
 (22, u'Rudolph', u'Giuliani', u'', u'R'),
 (24, u'Mike', u'Gravel', u'', u'D'),
 (26, u'John', u'Edwards', u'', u'D'),
 (29, u'Bill', u'Richardson', u'', u'D'),
 (30, u'Duncan', u'Hunter', u'', u'R'),
 (31, u'Dennis', u'Kucinich', u'', u'D'),
 (32, u'Ron', u'Paul', u'', u'R'),
 (33, u'Joseph', u'Biden', u'', u'D'),
 (34, u'Hillary', u'Clinton', u'R.', u'D'),
 (35, u'Mitt', u'Romney', u'', u'R'),
 (36, u'Samuel', u'Brownback', u'', u'R'),
 (37, u'John', u'McCain', u'', u'R'),
 (38, u'Tom', u'Tancredo', u'', u'R'),
 (39, u'Christopher', u'Dodd', u'J.', u'D'),
 (41, u'Fred', u'Thompson', u'D.', u'R')]

In [25]:
rem="""
DELETE FROM candidates;
"""
c = db.cursor().execute(rem)
db.commit()

In [26]:
%%bash
tail -n +2 candidates.txt > candidates_nohead.txt
echo ".import candidates_nohead_txt candidates" | sqlite3 cancont.db

Couldn't find program: u'bash'


In [27]:
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16


In [30]:
dfcwci.query("state=='VA' & amount < 400")

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [31]:
dfcwci[(dfcwci.state=='VA')&(dfcwci.amount<400)]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [32]:
cont_cols = [e[1] for e in make_query("PRAGMA table_info(contributors);")]

In [33]:
cont_cols

[u'id',
 u'last_name',
 u'first_name',
 u'middle_name',
 u'street_1',
 u'street_2',
 u'city',
 u'state',
 u'zip',
 u'amount',
 u'date',
 u'candidate_id']

In [43]:
def make_frame(list_of_tuples, legend=cont_cols):
    framelist=[]
    for i, cname in enumerate(legend):
        framelist.append((cname,[e[i] for e in list_of_tuples]))
        
    return pd.DataFrame.from_items(framelist)

In [44]:
out = make_query("SELECT * FROM contributors WHERE state='VA' AND amount < 400;")
make_frame(out)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
1,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
2,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32
3,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [46]:
out = make_query("SELECT * FROM contributors WHERE state IS NULL;")
make_frame(out)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,126,BOURNE,TRAVIS,None,LAGE KAART 77,None,BRASSCHATT,None,2930,-500,2008-11-20,35


In [48]:
dfcwci[dfcwci.state.isnull()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500,2008-11-20,35


In [49]:
out = make_query("SELECT * FROM contributors WHERE state IS NOT NULL;")
make_frame(out).shape

(174, 12)

In [50]:
dfcwci[dfcwci.state.notnull()].shape

(174, 11)

In [52]:
out=make_query("SELECT * FROM contributors WHERE state IN ('VA','WA');")
make_frame(out).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.00,2007-06-30,16
1,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
2,63,BURKE,SUZANNE,M.,3401 EVANSTON,None,SEATTLE,WA,981038677,-700.00,2008-03-05,22
3,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
4,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32
5,101,Aaronson,Rebecca,None,2000 Village Green Dr Apt 12,None,Mill Creek,WA,980125787,100.00,2008-02-08,34
6,107,Aaronson,Rebecca,None,2000 Village Green Dr Apt 12,None,Mill Creek,WA,980125787,100.00,2008-02-14,34
7,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [53]:
dfcwci[dfcwci.state.isin(['VA','WA'])].head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.00,2007-06-30,16
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
62,BURKE,SUZANNE,M.,3401 EVANSTON,NaN,SEATTLE,WA,981038677,-700.00,2008-03-05,22
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
100,Aaronson,Rebecca,NaN,2000 Village Green Dr Apt 12,NaN,Mill Creek,WA,980125787,100.00,2008-02-08,34
106,Aaronson,Rebecca,NaN,2000 Village Green Dr Apt 12,NaN,Mill Creek,WA,980125787,100.00,2008-02-14,34
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [54]:
out = make_query("SELECT * FROM contributors WHERE amount BETWEEN 10 AND 50;")
make_frame(out).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,3,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,50,2007-06-18,16
1,19,Ardle,William,None,412 Dakota Avenue,None,Springfield,OH,45504,50,2007-06-28,16
2,26,Buckler,Steve,None,24351 Armada Dr,None,Dana Point,CA,926291306,50,2007-07-30,20
3,27,Buckler,Steve,None,24351 Armada Dr,None,Dana Point,CA,926291306,25,2007-08-16,20
4,35,Buck,Barbara,None,1780 NE 138th St,None,North Miami,FL,331811316,50,2007-09-13,20
5,36,Buck,Barbara,None,1780 NE 138th St,None,North Miami,FL,331811316,50,2007-07-19,20
6,39,Buchanek,Elizabeth,None,7917 Kentbury Dr,None,Bethesda,MD,208144615,50,2007-09-30,20
7,50,Harrison,Ryan,None,2247 3rd St,None,La Verne,CA,917504918,25,2007-07-26,20
8,102,Aarons,Elaine,None,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25,2008-02-26,34
9,105,Aaron,Shirley,None,101 Cherry Ave,None,Havana,FL,323331311,50,2008-02-29,34


In [55]:
dfcwci.query("10<=amount<=50").head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16
18,Ardle,William,NaN,412 Dakota Avenue,NaN,Springfield,OH,45504,50,2007-06-28,16
25,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,50,2007-07-30,20
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25,2007-08-16,20
34,Buck,Barbara,NaN,1780 NE 138th St,NaN,North Miami,FL,331811316,50,2007-09-13,20
35,Buck,Barbara,NaN,1780 NE 138th St,NaN,North Miami,FL,331811316,50,2007-07-19,20
38,Buchanek,Elizabeth,NaN,7917 Kentbury Dr,NaN,Bethesda,MD,208144615,50,2007-09-30,20
49,Harrison,Ryan,NaN,2247 3rd St,NaN,La Verne,CA,917504918,25,2007-07-26,20
101,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25,2008-02-26,34
104,Aaron,Shirley,NaN,101 Cherry Ave,NaN,Havana,FL,323331311,50,2008-02-29,34


In [56]:
dfcwci.sort("amount").head(10)

C:\Users\Kevin\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
90,Kazor,Christopher,M,707 Spindletree ave,NaN,Naperville,IL,60565,-2592,2008-04-21,32
72,BRUNO,JOHN,NaN,10136 WINDERMERE CHASE BLVD.,NaN,GOTHA,FL,347344707,-2300,2008-03-06,22
64,BURKE,DONALD,J.,12 LOMPOC,NaN,RANCHO SANTA MARGA,CA,926881817,-2300,2008-03-11,22
73,BRUNO,IRENE,NaN,10136 WINDERMERE CHASE BLVD.,NaN,GOTHA,FL,347344707,-2300,2008-03-06,22
74,BROWN,TIMOTHY,J.,26826 MARLOWE COURT,NaN,STEVENSON RANCH,CA,913811020,-2300,2008-03-06,22
58,BURTON,GLENN,M.,4404 CHARLESTON COURT,NaN,TAMPA,FL,336092620,-2300,2008-03-05,22
57,BURTON,STEVEN,G.,9938 DEER CREEK DRIVE,NaN,TAMPA,FL,33647,-2300,2008-03-05,22
84,Uihlein,Richard,NaN,1396 N Waukegan Rd,NaN,Lake Forest,IL,600451147,-2300,2008-04-21,32
56,BURTON,SUSAN,NaN,9338 DEER CREEK DRIVE,NaN,TAMPA,FL,336472286,-2300,2008-03-05,22
55,BUSH,ERIC,NaN,P.O. BOX 61046,NaN,DENVER,CO,802061046,-2300,2008-03-06,22


In [58]:
dfcwci.sort_values(by="amount",ascending=False).head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600,2007-08-14,20
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600,2008-01-25,37
15,Anthony,John,NaN,211 Long Island Drive,NaN,Hot Springs,AR,71913,2300,2007-06-12,16
33,Buck,Blaine,M,45 Eaton Ave,NaN,Camden,ME,48431752,2300,2007-09-30,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300,2007-08-14,20
21,Baker,David,NaN,2550 Adamsbrooke Drive,NaN,Conway,AR,72034,2300,2007-04-11,16
13,Altes,R.D.,NaN,8600 Moody Road,NaN,Fort Smith,AR,72903,2300,2007-06-21,16
135,ABRAMOWITZ,NIRA,NaN,411 HARBOR ROAD,NaN,SOUTHPORT,CT,68901376,2300,2007-09-14,35
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500,2007-05-18,16
174,ABRAHAM,SALEM,A.,P.O. BOX 7,NaN,CANADIAN,TX,790140007,1300,2008-01-30,37


In [60]:
out = make_query("SELECT * FROM contributors ORDER BY amount")
make_frame(out).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,91,Kazor,Christopher,M,707 Spindletree ave,None,Naperville,IL,60565,-2592,2008-04-21,32
1,30,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,-2300,2007-08-14,20
2,52,BYINGTON,MARGARET,E.,2633 MIDDLEBORO LANE N.E.,None,GRAND RAPIDS,MI,495061254,-2300,2008-03-03,22
3,53,BYERS,BOB,A.,13170 TELFAIR AVENUE,None,SYLMAR,CA,913423573,-2300,2008-03-07,22
4,55,BUSH,KRYSTIE,None,P.O. BOX 61046,None,DENVER,CO,802061046,-2300,2008-03-06,22
5,56,BUSH,ERIC,None,P.O. BOX 61046,None,DENVER,CO,802061046,-2300,2008-03-06,22
6,57,BURTON,SUSAN,None,9338 DEER CREEK DRIVE,None,TAMPA,FL,336472286,-2300,2008-03-05,22
7,58,BURTON,STEVEN,G.,9938 DEER CREEK DRIVE,None,TAMPA,FL,33647,-2300,2008-03-05,22
8,59,BURTON,GLENN,M.,4404 CHARLESTON COURT,None,TAMPA,FL,336092620,-2300,2008-03-05,22
9,65,BURKE,DONALD,J.,12 LOMPOC,None,RANCHO SANTA MARGA,CA,926881817,-2300,2008-03-11,22


In [61]:
out = make_query("SELECT * FROM contributors ORDER BY amount DESC;")
make_frame(out).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,31,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600,2007-08-14,20
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,None,MIAMI SPRINGS,FL,331663832,2600,2008-01-25,37
2,14,Altes,R.D.,None,8600 Moody Road,None,Fort Smith,AR,72903,2300,2007-06-21,16
3,16,Anthony,John,None,211 Long Island Drive,None,Hot Springs,AR,71913,2300,2007-06-12,16
4,22,Baker,David,None,2550 Adamsbrooke Drive,None,Conway,AR,72034,2300,2007-04-11,16
5,29,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,2300,2007-08-14,20
6,34,Buck,Blaine,M,45 Eaton Ave,None,Camden,ME,48431752,2300,2007-09-30,20
7,136,ABRAMOWITZ,NIRA,None,411 HARBOR ROAD,None,SOUTHPORT,CT,68901376,2300,2007-09-14,35
8,6,Akin,Mike,None,181 Baywood Lane,None,Monticello,AR,71655,1500,2007-05-18,16
9,10,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1300,2007-06-29,16


In [62]:
dfcwci[['first_name','amount']].head(10)

,first_name,amount
0,Steven,500
1,Don,250
2,Don,50
3,Don,100
4,Charles,100
5,Mike,1500
6,Rebecca,500
7,Brittni,250
8,John D.,1000
9,John D.,1300


In [64]:
out=make_query("SELECT first_name, amount FROM contributors;")
make_frame(out,['first_name','amount']).head(10)

,first_name,amount
0,Steven,500
1,Don,250
2,Don,50
3,Don,100
4,Charles,100
5,Mike,1500
6,Rebecca,500
7,Brittni,250
8,John D.,1000
9,John D.,1300


In [69]:
dfcwci[['last_name','first_name']].count()

last_name     175
first_name    175
dtype: int64

In [70]:
dfcwci[['last_name','first_name']].drop_duplicates().count()

last_name     126
first_name    126
dtype: int64

In [72]:
out = make_query("SELECT DISTINCT last_name, first_name FROM contributors;")
make_frame(out,['last_name','first_name']).head(10)

,last_name,first_name
0,AARONS,CHARLES
1,ABACHERLI,SHIRLEY
2,ABAIR,PETER
3,ABATE,MARIA
4,ABBO,PAULINE
5,ABBOT,DAVID
6,ABBOTT,GERALD
7,ABBOTT,MIKE
8,ABBOTT,ROBERT
9,ABBOTT,RONALD


In [77]:
dfcwci['name'] = dfcwci["last_name"]+", "+dfcwci["first_name"]
dfcwci.head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16,"Agee, Steven"
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16,"Ahrens, Don"
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16,"Ahrens, Don"
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles"
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500,2007-05-18,16,"Akin, Mike"
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500,2007-05-18,16,"Akin, Rebecca"
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250,2007-06-06,16,"Aldridge, Brittni"
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000,2007-06-11,16,"Allen, John D."
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300,2007-06-29,16,"Allen, John D."


In [78]:
dfcwci.assign(ucname=dfcwci.last_name+":"+dfcwci.first_name).head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name,ucname
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16,"Agee, Steven",Agee:Steven
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16,"Ahrens, Don",Ahrens:Don
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16,"Ahrens, Don",Ahrens:Don
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16,"Ahrens, Don",Ahrens:Don
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles",Akin:Charles
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500,2007-05-18,16,"Akin, Mike",Akin:Mike
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500,2007-05-18,16,"Akin, Rebecca",Akin:Rebecca
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250,2007-06-06,16,"Aldridge, Brittni",Aldridge:Brittni
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000,2007-06-11,16,"Allen, John D.",Allen:John D.
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300,2007-06-29,16,"Allen, John D.",Allen:John D.


In [79]:
dfcwci

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16,"Agee, Steven"
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16,"Ahrens, Don"
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16,"Ahrens, Don"
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles"
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500,2007-05-18,16,"Akin, Mike"
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500,2007-05-18,16,"Akin, Rebecca"
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250,2007-06-06,16,"Aldridge, Brittni"
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000,2007-06-11,16,"Allen, John D."
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300,2007-06-29,16,"Allen, John D."


In [81]:
dfcwci[dfcwci.state=='VA']

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.00,2007-06-30,16,"Agee, Steven"
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20,"Buckheit, Bruce"
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32,"Ranganath, Anoop"
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32,"Perreault, Louise"
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35,"ABDELLA, THOMAS"


In [82]:
dfcwci.loc[dfcwci.state=='VA','name']

0           Agee, Steven
27       Buckheit, Bruce
77      Ranganath, Anoop
88     Perreault, Louise
145      ABDELLA, THOMAS
Name: name, dtype: object

In [84]:
dfcwci.loc[dfcwci.state=='VA','name']="junk"

In [85]:
dfcwci.query("state=='VA'")

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.00,2007-06-30,16,junk
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20,junk
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32,junk
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32,junk
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35,junk


In [87]:
alt="ALTER TABLE contributors ADD COLUMN name;"
db.cursor().execute(alt)

In [88]:
make_query("PRAGMA table_info(contributors);")

[(0, u'id', u'INTEGER', 1, None, 1),
 (1, u'last_name', u'VARCHAR', 0, None, 0),
 (2, u'first_name', u'VARCHAR', 0, None, 0),
 (3, u'middle_name', u'VARCHAR', 0, None, 0),
 (4, u'street_1', u'VARCHAR', 0, None, 0),
 (5, u'street_2', u'VARCHAR', 0, None, 0),
 (6, u'city', u'VARCHAR', 0, None, 0),
 (7, u'state', u'VARCHAR', 0, None, 0),
 (8, u'zip', u'VARCHAR', 0, None, 0),
 (9, u'amount', u'INTEGER', 0, None, 0),
 (10, u'date', u'DATETIME', 0, None, 0),
 (11, u'candidate_id', u'INTEGER', 1, None, 0),
 (12, u'name', u'', 0, None, 0)]

In [90]:
out = make_query("SELECT id, last_name,first_name from contributors;")
out2 = [(e[1]+", "+e[2],e[0]) for e in out]
out2

[(u'Agee, Steven', 1),
 (u'Ahrens, Don', 2),
 (u'Ahrens, Don', 3),
 (u'Ahrens, Don', 4),
 (u'Akin, Charles', 5),
 (u'Akin, Mike', 6),
 (u'Akin, Rebecca', 7),
 (u'Aldridge, Brittni', 8),
 (u'Allen, John D.', 9),
 (u'Allen, John D.', 10),
 (u'Allison, John W.', 11),
 (u'Allison, Rebecca', 12),
 (u'Allison, Rebecca', 13),
 (u'Altes, R.D.', 14),
 (u'Andres, Dale', 15),
 (u'Anthony, John', 16),
 (u'Arbogast, Robert', 17),
 (u'Arbogast, Robert', 18),
 (u'Ardle, William', 19),
 (u'Atiq, Omar', 20),
 (u'Atiq, Omar', 21),
 (u'Baker, David', 22),
 (u'Bancroft, David', 23),
 (u'Banks, Charles', 24),
 (u'Barbee, John', 25),
 (u'Buckler, Steve', 26),
 (u'Buckler, Steve', 27),
 (u'Buckheit, Bruce', 28),
 (u'Buckel, Linda', 29),
 (u'Buckel, Linda', 30),
 (u'Buckel, Linda', 31),
 (u'Buck, Thomas', 32),
 (u'Buck, Jay', 33),
 (u'Buck, Blaine', 34),
 (u'Buck, Barbara', 35),
 (u'Buck, Barbara', 36),
 (u'Buchman, Mark M', 37),
 (u'Bucher, Ida', 38),
 (u'Buchanek, Elizabeth', 39),
 (u'Buchanan, John', 40),


In [91]:
alt2="UPDATE contributors SET name = ? WHERE id = ?;"
for ele in out2:
    db.cursor().execute(alt2, ele)

In [92]:
db.commit()

In [93]:
out = make_query("SELECT * from contributors;")
make_frame(out,cont_cols+["name"]).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500,2007-06-30,16,"Agee, Steven"
1,2,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,250,2007-05-16,16,"Ahrens, Don"
2,3,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,50,2007-06-18,16,"Ahrens, Don"
3,4,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,100,2007-06-21,16,"Ahrens, Don"
4,5,Akin,Charles,None,10187 Sugar Creek Road,None,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles"
5,6,Akin,Mike,None,181 Baywood Lane,None,Monticello,AR,71655,1500,2007-05-18,16,"Akin, Mike"
6,7,Akin,Rebecca,None,181 Baywood Lane,None,Monticello,AR,71655,500,2007-05-18,16,"Akin, Rebecca"
7,8,Aldridge,Brittni,None,"808 Capitol Square Place, SW",None,Washington,DC,20024,250,2007-06-06,16,"Aldridge, Brittni"
8,9,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1000,2007-06-11,16,"Allen, John D."
9,10,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1300,2007-06-29,16,"Allen, John D."


In [95]:
upd="UPDATE contributors SET name = 'junk' WHERE state = 'VA';"
db.cursor().execute(upd)
db.commit()

In [96]:
out=make_query("SELECT * from contributors where state='VA';")
make_frame(out,cont_cols+["name"]).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.00,2007-06-30,16,junk
1,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20,junk
2,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32,junk
3,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32,junk
4,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35,junk


In [97]:
dfcwci.describe()

,zip,amount,candidate_id
count,1.750000e+02,175.000000,175.000000
mean,3.780014e+08,3.418114,28.000000
std,3.628278e+08,1028.418999,7.823484
min,2.474000e+03,-2592.000000,16.000000
25%,9.336700e+04,-175.000000,20.000000
50%,3.233313e+08,100.000000,32.000000
75%,7.816946e+08,300.000000,35.000000
max,9.951532e+08,4600.000000,37.000000


In [98]:
dfcwci.amount.max()

4600.0

In [99]:
dfcwci[dfcwci.amount==dfcwci.amount.max()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600,2007-08-14,20,"Buckel, Linda"


In [100]:
out=make_query("SELECT *, MAX(amount) AS maxamt FROM contributors;")
print out
make_frame(out,cont_cols+["maxamt"])

[(175, u'ABRAHAM', u'SALEM', u'A.', u'P.O. BOX 7', None, u'CANADIAN', u'TX', u'790140007', 1300, u'2008-01-30', 37, u'ABRAHAM, SALEM', 4600)]


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,maxamt
0,175,ABRAHAM,SALEM,A.,P.O. BOX 7,None,CANADIAN,TX,790140007,1300,2008-01-30,37,"ABRAHAM, SALEM"


In [101]:
out=make_query("SELECT COUNT(amount) AS AMOUNTCOUNT FROM contributors;")
print out

[(175,)]


In [102]:
out=make_query("SELECT AVG(amount) FROM contributors")
print out

[(3.418114285714276,)]


In [103]:
dfcwci.groupby("state").sum()

,zip,amount,candidate_id
state,,,
AK,2985459621,1210.00,111
AR,864790,14200.00,192
AZ,860011121,120.00,37
CA,14736360720,-5013.73,600
CO,2405477834,-5823.00,111
CT,68901376,2300.00,35
DC,800341853,-1549.91,102
FL,8970626520,-4050.00,803
IA,50266,250.00,16


In [104]:
dfcwci.groupby("state")['amount'].mean()

state
AK     403.333333
AR    1183.333333
AZ     120.000000
CA    -217.988261
CO   -1455.750000
CT    2300.000000
DC    -309.982000
FL    -135.000000
IA     250.000000
ID    -261.000000
IL    -931.133333
KS    -330.000000
KY    -200.000000
LA     650.000000
MA     -13.833333
MD     150.000000
ME     630.000000
MI    -253.000000
MN     107.333333
MO     100.000000
NC     500.000000
NH     -24.600000
NJ    -408.725000
NV     181.250000
NY    -809.312500
OH     112.500000
OK     266.666667
PA    -429.200000
RI     100.000000
SC     800.000000
TN     -25.000000
TX     220.582222
UT     459.090909
VA     103.184000
WA    -166.666667
Name: amount, dtype: float64

In [105]:
dfcwci.state.unique()

array(['VA', 'CA', 'AR', 'DC', 'SC', 'IA', 'OH', 'NC', 'UT', 'MO', 'IL',
       'ME', 'FL', 'MD', 'MI', 'CO', 'WA', 'NY', 'TX', 'KY', 'PA', 'TN',
       'MA', 'MN', 'KS', 'NJ', 'NH', 'ID', 'OK', nan, 'NV', 'CT', 'RI',
       'AK', 'LA', 'AZ'], dtype=object)

In [106]:
out=make_query("SELECT state,SUM(amount) FROM contributors GROUP BY state;")
make_frame(out,legend=['state','sum'])

,state,sum
0,None,-500.00
1,AK,1210.00
2,AR,14200.00
3,AZ,120.00
4,CA,-5013.73
5,CO,-5823.00
6,CT,2300.00
7,DC,-1549.91
8,FL,-4050.00
9,IA,250.00


In [107]:
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16,junk
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16,"Ahrens, Don"
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16,"Ahrens, Don"
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles"


In [109]:
df2 = dfcwci.copy()
df2.set_index('last_name', inplace=True)
df2.head()

,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
last_name,,,,,,,,,,,
Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16,junk
Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16,"Ahrens, Don"
Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16,"Ahrens, Don"
Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16,"Ahrens, Don"
Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles"


In [110]:
df2.drop(['Ahrens'],inplace=True)
df2.head()

,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
last_name,,,,,,,,,,,
Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16,junk
Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles"
Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500,2007-05-18,16,"Akin, Mike"
Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500,2007-05-18,16,"Akin, Rebecca"
Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250,2007-06-06,16,"Aldridge, Brittni"


In [113]:
df2.reset_index(inplace=True)
df2.head()

,index,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16,junk
1,1,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles"
2,2,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500,2007-05-18,16,"Akin, Mike"
3,3,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500,2007-05-18,16,"Akin, Rebecca"
4,4,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250,2007-06-06,16,"Aldridge, Brittni"


In [115]:
out = make_query("SELECT * FROM contributors LIMIT 3;")
make_frame(out).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500,2007-06-30,16
1,2,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,250,2007-05-16,16
2,3,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,50,2007-06-18,16


In [116]:
dfcwci[0:3]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16,junk
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16,"Ahrens, Don"


In [117]:
crind="CREATE INDEX amount_ix ON contributors(amount);"
db.cursor().execute(crind)
db.commit()

In [9]:
dfcand.head()

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D


In [15]:
obamaid=dfcand.query("last_name=='Obama'")["id"].values[0]

In [17]:
obamacontrib=dfcwci.query("candidate_id==%i" % obamaid)
obamacontrib.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,33
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,33
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,33
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,33
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,33


In [ ]:
russiandollsel="""
SELECT * FROM contributors WHERE
    candidate_id = (SELECT id from candidates WHERE last_name = 'Obama');
"""